# Contextual Risk Expert Model
Reference: traffic-accident-analytics-ml + featureengineering-safedrive

This notebook implements contextual risk assessment based on time, weather, and traffic conditions.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

print("Contextual Risk Expert Model - Ready for implementation")

## Temporal Risk Factors

Key factors:
- Time of day
- Day of week
- Holiday periods
- Rush hour patterns

In [ ]:
def extract_temporal_features(timestamp):
    """
    Extract temporal features from timestamp
    """
    features = {}
    
    dt = pd.to_datetime(timestamp)
    
    features['hour'] = dt.hour
    features['day_of_week'] = dt.dayofweek
    features['month'] = dt.month
    features['is_weekend'] = dt.dayofweek >= 5
    features['is_rush_hour'] = (7 <= dt.hour <= 9) or (17 <= dt.hour <= 19)
    features['is_night'] = (dt.hour >= 22) or (dt.hour <= 6)
    
    return features

# Test temporal features
test_features = extract_temporal_features('2023-07-15 18:30:00')
print(f"Sample temporal features: {test_features}")

## Weather Risk Assessment

Weather conditions impact on driving risk

In [ ]:
def calculate_weather_risk(weather_data):
    """
    Calculate weather-based risk score
    """
    base_risk = 20
    
    # Precipitation impact
    precipitation = weather_data.get('precipitation_mm', 0)
    if precipitation > 0:
        base_risk += min(30, precipitation * 2)
    
    # Temperature impact (ice/snow conditions)
    temperature = weather_data.get('temperature_c', 20)
    if temperature < 0:
        base_risk += 25
    elif temperature < 5:
        base_risk += 15
    
    # Visibility impact
    visibility = weather_data.get('visibility_km', 10)
    if visibility < 5:
        base_risk += 20
    elif visibility < 10:
        base_risk += 10
    
    # Wind impact
    wind_speed = weather_data.get('wind_speed_kmh', 0)
    if wind_speed > 50:
        base_risk += 15
    
    return min(100, base_risk)

# Test weather risk
test_weather = {
    'precipitation_mm': 5,
    'temperature_c': 2,
    'visibility_km': 8,
    'wind_speed_kmh': 25
}
weather_risk = calculate_weather_risk(test_weather)
print(f"Weather risk score: {weather_risk}")

## Traffic Conditions Risk

Traffic density and flow impact on risk

In [ ]:
def calculate_traffic_risk(traffic_data):
    """
    Calculate traffic-based risk score
    """
    base_risk = 25
    
    # Traffic density impact
    density = traffic_data.get('density', 'normal')
    density_multipliers = {
        'light': 0.8,
        'normal': 1.0,
        'moderate': 1.3,
        'heavy': 1.6
    }
    base_risk *= density_multipliers.get(density, 1.0)
    
    # Speed flow analysis
    avg_speed = traffic_data.get('average_speed_kmh', 50)
    speed_limit = traffic_data.get('speed_limit_kmh', 50)
    
    speed_ratio = avg_speed / speed_limit
    if speed_ratio < 0.5:  # Very slow traffic
        base_risk *= 1.2
    elif speed_ratio > 1.3:  # Very fast traffic
        base_risk *= 1.4
    
    # Active incidents
    incidents = traffic_data.get('active_incidents', 0)
    base_risk += min(20, incidents * 5)
    
    return min(100, base_risk)

# Test traffic risk
test_traffic = {
    'density': 'heavy',
    'average_speed_kmh': 25,
    'speed_limit_kmh': 60,
    'active_incidents': 2
}
traffic_risk = calculate_traffic_risk(test_traffic)
print(f"Traffic risk score: {traffic_risk}")

## Combined Contextual Model

Combine all contextual factors into a single risk score

In [ ]:
def calculate_contextual_risk(timestamp, weather_data, traffic_data):
    """
    Calculate comprehensive contextual risk score
    """
    # Get individual risk components
    temporal_features = extract_temporal_features(timestamp)
    weather_risk = calculate_weather_risk(weather_data)
    traffic_risk = calculate_traffic_risk(traffic_data)
    
    # Base temporal risk
    temporal_risk = 30
    
    # Time-based adjustments
    if temporal_features['is_rush_hour']:
        temporal_risk += 20
    if temporal_features['is_night']:
        temporal_risk += 15
    if temporal_features['is_weekend'] and temporal_features['is_night']:
        temporal_risk += 25  # Weekend night driving
    
    # Weighted combination
    combined_risk = (
        temporal_risk * 0.3 +
        weather_risk * 0.4 +
        traffic_risk * 0.3
    )
    
    return {
        'overall_risk': min(100, combined_risk),
        'temporal_risk': temporal_risk,
        'weather_risk': weather_risk,
        'traffic_risk': traffic_risk,
        'temporal_features': temporal_features
    }

# Test combined contextual risk
contextual_result = calculate_contextual_risk(
    '2023-07-15 18:30:00',
    test_weather,
    test_traffic
)
print(f"Contextual risk assessment: {contextual_result}")